# 사전 전처리

숙박/관광지/F&B 별 데이터셋을 만들기 위해 지번주소 단위 온실가스 배출량 데이터(TB_ECO_BUILDING_GHG_GIS_JEJU_1920)와 비짓제주 조회수 데이터(JT_MT_ACCTO_TRRSRT_SCCNT_LIST)를 전처리하는 코드입니다.

1. 지번주소 단위 온실가스 배출 데이터((TB_ECO_BUILDING_GHG_GIS_JEJU_1920) 전처리
 - 월별 탄소배출량 변수 생성
 - 월별 일자로 나누어 일별 탄소배출량으로 계산
 - 지번주소별 탄소배출량 평균/최댓값 계산
   
  
 2. 비짓제주 조회수 데이터(JT_MT_ACCTO_TRRSRT_SCCNT_LIST) 전처리
 - 0으로 채워진 결측치 NA로 변경
 - 이름 전처리
 - 월별 조회 수가 다른 행으로 나눠져 있는 경우 정리


In [1]:
import pandas as pd
import requests
import logging
from tqdm import tqdm

## 1. 온실가스 배출량 데이터 전처리


### 1-(1) 월별 배출량 변수 생성

In [2]:
data = pd.read_csv('data/TB_ECO_BUILDING_GHG_GIS_JEJU_1920.csv', encoding='cp949')

In [3]:
data = data.groupby(['LTNO_ADDR', 'USE_YR', 'USE_MONTH']).sum().reset_index()
data['YEAR_MONTH'] = data['USE_YR'].apply(str) + '_' + data['USE_MONTH'].apply(str)

In [4]:
data = data[['LTNO_ADDR', 'GPS_LNGTD', 'GPS_LTTD', 'SUM_GRGS_DSAMT', 'YEAR_MONTH']]

In [5]:
data

,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,SUM_GRGS_DSAMT,YEAR_MONTH
0,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.000000,2019_3
1,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.000000,2019_4
2,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.000000,2019_5
3,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.000000,2019_9
4,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.000000,2019_10
...,...,...,...,...,...
329360,제주특별자치도 제주시 회천동 79번지,0.0,0.0,1.268667,2019_11
329361,제주특별자치도 제주시 회천동 79번지,0.0,0.0,1.315292,2020_3
329362,제주특별자치도 제주시 회천동 79번지,0.0,0.0,1.375905,2020_4
329363,제주특별자치도 제주시 회천동 79번지,0.0,0.0,1.264005,2020_5


In [6]:
data_unique = data[['LTNO_ADDR', 'GPS_LNGTD', 'GPS_LTTD']].drop_duplicates().reset_index(drop=True) ; data_unique

,LTNO_ADDR,GPS_LNGTD,GPS_LTTD
0,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0
1,제주특별자치도 서귀포시 강정동 1026-1번지,0.0,0.0
2,제주특별자치도 서귀포시 강정동 1045-5번지,0.0,0.0
3,제주특별자치도 서귀포시 강정동 108-10번지,0.0,0.0
4,제주특별자치도 서귀포시 강정동 1155번지,0.0,0.0
...,...,...,...
34515,제주특별자치도 제주시 회천동 656-2번지,0.0,0.0
34516,제주특별자치도 제주시 회천동 722-2번지,0.0,0.0
34517,제주특별자치도 제주시 회천동 742-2번지,0.0,0.0
34518,제주특별자치도 제주시 회천동 79-2번지,0.0,0.0


In [7]:
data_unique[['2019_3', '2019_4', '2019_5', '2019_9', '2019_10', '2019_11', '2020_3', '2020_4', '2020_5', '2020_9', '2020_10']]= 0.0

In [8]:
for value, yr_mt,add in zip(data['SUM_GRGS_DSAMT'],data['YEAR_MONTH'], data['LTNO_ADDR']):
    du_idx = data_unique[data_unique['LTNO_ADDR'] == add].index[0]
    data_unique.at[du_idx, yr_mt] = value

In [9]:
data_unique = data_unique[['LTNO_ADDR', 'GPS_LNGTD', 'GPS_LTTD', '2019_3', '2019_4', '2019_5','2019_9', '2019_10', '2019_11', '2020_3', '2020_4', '2020_5', '2020_9', '2020_10']]

### 1-(2) 일별 데이터로 계산
월별 일자 수로 각 월별 탄소배출량 나누기

In [10]:
data = data_unique.copy()

In [11]:
data['2019_3'] = data['2019_3']/31
data['2019_4'] = data['2019_4']/30
data['2019_5'] = data['2019_5']/31
data['2019_9'] = data['2019_9']/30
data['2019_10'] = data['2019_10']/31
data['2019_11'] = data['2019_11']/30
data['2020_3'] = data['2020_3']/31
data['2020_4'] = data['2020_4']/30
data['2020_5'] = data['2020_5']/31
data['2020_9'] = data['2020_9']/30
data['2020_10'] = data['2020_10']/31

### 1-(3) 지번주소별 탄소배출량 평균/최댓값 계산

In [12]:
data['co2_mean'] = [ data.iloc[idx][data.columns[-11:]].mean() for idx in range(len(data))]
data['co2_max'] = [ data.iloc[idx][data.columns[-11:]].max() for idx in range(len(data))]

In [13]:
data = data.drop(['2019_3', '2019_4', '2019_5', '2019_9', '2019_10', '2019_11', '2020_3', '2020_4', '2020_5', '2020_9', '2020_10'], axis=1)
data.to_csv('data/PRE_CO2_지번단위탄소배출량_meanmax.csv', encoding='utf-8-sig', index=False)

In [14]:
data

,LTNO_ADDR,GPS_LNGTD,GPS_LTTD,co2_mean,co2_max
0,제주특별자치도 서귀포시 강정동 0번지,0.0,0.0,0.002794,0.009123
1,제주특별자치도 서귀포시 강정동 1026-1번지,0.0,0.0,0.071235,0.109615
2,제주특별자치도 서귀포시 강정동 1045-5번지,0.0,0.0,0.000990,0.000990
3,제주특별자치도 서귀포시 강정동 108-10번지,0.0,0.0,0.031012,0.054986
4,제주특별자치도 서귀포시 강정동 1155번지,0.0,0.0,0.001266,0.002114
...,...,...,...,...,...
34515,제주특별자치도 제주시 회천동 656-2번지,0.0,0.0,0.048000,0.080382
34516,제주특별자치도 제주시 회천동 722-2번지,0.0,0.0,0.001769,0.005129
34517,제주특별자치도 제주시 회천동 742-2번지,0.0,0.0,0.000784,0.002098
34518,제주특별자치도 제주시 회천동 79-2번지,0.0,0.0,0.005633,0.031270


## 2. 비짓제주 조회수 데이터 전처리

In [15]:
import pandas as pd
import numpy as np
import re

In [16]:
def name_preprocessing(name):
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    name = name.split('(')[0]
    return name.strip()

In [17]:
def vj_grouping(x):
    return pd.Series({'AREA_NM' : x['AREA_NM'].unique(),
                      'ADDR' : x['ADDR'].unique(),
                      'BASE_YEAR' : x['BASE_YEAR'].sum(),
                      'MAR_VIEW_CO' : int(x['MAR_VIEWS_CO'].sum()),
                      'APR_VIEW_CO' : int(x['APR_VIEWS_CO'].sum()),
                      'MAY_VIEW_CO' : int(x['MAY_VIEWS_CO'].sum()),
                      'SEP_VIEW_CO' : int(x['SEP_VIEWS_CO'].sum()),
                      'OCT_VIEW_CO' : int(x['OCT_VIEWS_CO'].sum()),
                      'NOV_VIEW_CO' : int(x['NOV_VIEWS_CO'].sum())})

In [18]:
vj_view = pd.read_csv('data/JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv', encoding='cp949')

In [19]:
# 0.0으로 채워져 있는 행들을 다 NA로 바꿈
for hmm in vj_view.columns:
    vj_view[f'{hmm}'] = vj_view[f'{hmm}'].replace(0.0, np.nan)

In [20]:
vj_view

,CL_CD,CL_NM,AREA_NM,ADDR,BASE_YEAR,ALL_TOTAL_CO,JAN_VIEWS_CO,FEB_VIEWS_CO,MAR_VIEWS_CO,APR_VIEWS_CO,MAY_VIEWS_CO,JUN_VIEWS_CO,JULY_VIEWS_CO,AUG_VIEWS_CO,SEP_VIEWS_CO,OCT_VIEWS_CO,NOV_VIEWS_CO,DEC_VIEWS_CO
0,c3,숙박,메리어트관 제주신화월드 호텔앤리조트,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38,2021,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,NaN,NaN,NaN
1,c3,숙박,1915 지오하우스,제주특별자치도 서귀포시 성산읍 성산등용로 14,2021,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN
2,c3,숙박,521 게스트 하우스,제주특별자치도 제주시 애월읍 중용길 52-1 (신엄리) 521 게스트 하우스,2021,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN
3,c3,숙박,가까이에파도소리,제주도 제주시 외도2동 연대마을길 44,2021,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN
4,c3,숙박,가름게스트하우스,제주특별자치도 서귀포시 법환하로9번길 10,2021,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22215,c4,음식점,흑돼지촌,제주특별자치도 제주시 한림읍 한림상로 62,2022,14.0,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22216,c4,음식점,흑돼지킹생구이전문점,제주특별자치도 제주시 도령로11길 26-9,2022,16.0,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22217,c4,음식점,흑소랑,제주특별자치도 제주시 연북로 631,2022,33.0,NaN,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22218,c4,음식점,흑염소마루,제주특별자치도 서귀포시 남원읍 태위로663번길 1-1,2022,30.0,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 이름 preprocessing -> 새로운 열 만듦
vj_view['AREA_NM_PRE'] = vj_view['AREA_NM'].apply(name_preprocessing)

In [22]:
# 3월/4월/5월/9월/10월/11월이 각각 다른 행으로 잡혀 있는 거 수정
vj_view = vj_view.groupby(['AREA_NM_PRE', 'CL_NM']).apply(vj_grouping)
vj_view = vj_view.reset_index() ; vj_view

,AREA_NM_PRE,CL_NM,AREA_NM,ADDR,BASE_YEAR,MAR_VIEW_CO,APR_VIEW_CO,MAY_VIEW_CO,SEP_VIEW_CO,OCT_VIEW_CO,NOV_VIEW_CO
0,1080칼국수,음식점,[1080칼국수],[제주특별자치도 제주시 서광로13길 3],12129,9,14,15,18,23,69
1,1100고지,관광지,[1100고지(휴게소)],[제주특별자치도 서귀포시 1100로 1555],12129,438,420,506,323,451,541
2,1100고지습지,관광지,[1100고지습지],[제주특별자치도 서귀포시 색달동 산 1-2],12129,613,561,608,494,624,960
3,1112도로,관광지,[1112도로],[제주특별자치도 제주시 명림로 584 (봉개동)],12129,60,53,53,109,119,112
4,1915지오하우스,숙박,[1915 지오하우스],[제주특별자치도 서귀포시 성산읍 성산등용로 14],12129,38,43,33,44,40,47
...,...,...,...,...,...,...,...,...,...,...,...
4034,흰수염고래리조트,숙박,[흰수염고래리조트],[제주특별자치도 제주시 애월읍 일주서로 6818],12129,10,28,20,46,64,59
4035,히든클리프호텔앤네이쳐,숙박,[히든클리프 호텔 앤 네이쳐],[제주특별자치도 서귀포시 예래해안로 542],12129,106,124,135,96,107,101
4036,힐링캠프펜션,숙박,[힐링캠프펜션],[서귀포시 남원읍 위미항구로 96],12129,12,12,12,21,27,20
4037,힐링하우스,숙박,[힐링하우스],[제주특별자치도 제주시 홍랑길 4-6],4042,0,0,0,0,0,0


In [24]:
vj_view.to_csv('data/PRE_JT_MT_ACCTO_TRRSRT_SCCNT_LIST.csv', encoding='utf-8-sig', index=False)